# Model training

This notebook takes numpy arrays prepared from ASTER satellite imagery using the same functions as are found in the module `array_processing`. Not found in that module is the simple function from the `rasterio` package that reads a geoTIFF file and returns a numpy array.

File names and labels of the data were previously generated as python lists and saved using pickle.

5% of the data has been withheld for final testing purposes.

## Imports

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from functools import partial
import numpy as np
import pickle
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Read in all the training data

use the command `!cp - r source_folder destination_folder` to copy all training arrays into colab session storage.

In [ ]:
file_names_path = "" # path to the pickle object containing the list of file names used in model development
with open(file_names_path, 'rb') as file:
    file_names = pickle.load(file)

In [ ]:
data_dir = "" # local folder containing training arrays

In [ ]:
file_paths = [os.path.join(data_dir, file_name) for file_name in file_names if file_name.endswith('.npy')]

In [ ]:
labels_path = "" # path to the pickle object containing the list of lables used in model development
with open(labels_path, 'rb') as file:
    labels = pickle.load(file)

## Prepare the tensorflow dataset

The numpy arrays have already been processed, but functions are needed to read them in based on the file names and to set their shape attribute for tensorflow

In [ ]:
def load_numpy_file(file_path, label):
    array = np.load(file_path, allow_pickle = True)
    array = np.ndarray.astype(array, np.float32)
    return array, label

def parse_function(file_path, label):
    array, label = tf.numpy_function(load_numpy_file, [file_path, label], [tf.float32, tf.int32])
    return array, label

def fixup_shape_11(images, labels):
    images.set_shape([224, 224, 11])
    labels.set_shape([])
    return images, labels

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))

In [ ]:
dataset = dataset.shuffle(buffer_size = len(dataset), seed = 42, reshuffle_each_iteration = False)

In [ ]:
dataset = dataset.map(parse_function)

In [ ]:
dataset = dataset.map(fixup_shape_11)

In [ ]:
train_size = round(len(dataset) * 0.8)
train_ds = dataset.take(train_size)
val_ds = dataset.skip(train_size)

bs = 8
train_ds = train_ds.batch(bs).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.batch(bs).prefetch(tf.data.AUTOTUNE)

## Set up the model

This model is a combination of the pre-trained [EffNetV2-XL(21k)](https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_xl/feature_vector/2) with several preceding layers that serve to reduce the original 11 bands to the expected 3 bands.

A DepthPool class is defined to pool the results of convolutions across bands.

In [ ]:
class DepthPool(tf.keras.layers.Layer):
    def __init__(self, pool_size = 2, **kwargs):
        super().__init__(**kwargs)
        self.pool_size = pool_size

    def call(self, inputs):
        shape = tf.shape(inputs)
        groups = shape[-1] // self.pool_size
        new_shape = tf.concat([shape[:-1], [groups, self.pool_size]], axis = 0)
        return tf.reduce_max(tf.reshape(inputs, new_shape), axis = -1)

In [ ]:
DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size = 3, padding = "same", activation = "relu", kernel_initializer = "he_normal")


In [ ]:
model_handle = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_xl/feature_vector/2"

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape = (224, 224, 11)),
    tf.keras.layers.Conv2D(filters = 64, kernel_size = (5, 5), strides = (1, 1), activation = 'relu'),
    DefaultConv2D(filters = 64),
    DepthPool(),
    DefaultConv2D(filters = 16),
    DefaultConv2D(filters = 8),
    DepthPool(),
    DefaultConv2D(filters = 3),

    hub.KerasLayer(model_handle, trainable=True),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(1,
                          kernel_regularizer=tf.keras.regularizers.l2(0.0001),
                          activation = 'sigmoid')
])
model.build((None,)+(224, 224)+(3,))

In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
CHECKPOINT_DIR = "" # File path for saving model checkpoints
callbacks = [
    ModelCheckpoint(CHECKPOINT_DIR, monitor='val_accuracy', save_weights_only = True, save_best_only= True, mode = 'max')
]

In [ ]:
compound_fitted = model.fit(train_ds, validation_data = val_ds, epochs = 200, callbacks = callbacks)

In [ ]:
model_save_path = "" # Folder in which to save the complete model
model.save(model_save_path)